In [1]:
import os

In [2]:
from pest_lookup import *


In [68]:
import time

In [3]:
os.chdir('pesticide_lists')

In [4]:
import pandas as pd

In [5]:
os.listdir()

['.RData',
 '.Rhistory',
 'all_epa.csv',
 'CAS_lookup.R',
 'chemical_ref_nums.txt',
 'chem_lookup.R',
 'eco_apple_banned.txt',
 'eco_apple_restricted.txt',
 'FlorVerde_banned_pesticides.txt',
 'FTUSA_banned.txt',
 'FTUSA_restricted.txt',
 'Pesticides Lists.xlsx',
 'restricted_pesticides.txt',
 'scratch',
 'SIP_banned_pesticides.txt',
 'TRUEarth_banned.txt',
 'trueEarth_restricted.txt']

In [6]:
import json
import requests
from bs4 import BeautifulSoup

In [79]:
with open('chemical_ref_nums.txt', encoding='utf-16') as file:
    dic=json.loads(file.read())

UnicodeError: UTF-16 stream does not start with BOM

In [81]:
'Pentachlorophenol\\xc2\\xa0and its salts and esters'.replace('\\xc2', '').replace('\\xa0', '')

'Pentachlorophenoland its salts and esters'

In [14]:
full_searcher('Heptabromdiphenyl ether')

Trying Wikipedia


('Heptabromdiphenyl ether', [])

In [86]:
def string_fix(stri):
    return  re.sub(r'\\x\w\d', '', stri)

In [85]:
  'Fosfamidon Formulaci\\xc3\\xb3n L\\xc3\\xadquida Soluble')

'Fosfamidon Formulacin L\\xadquida Soluble'

In [12]:
[k for k,v in dic.items() if v==[]]

['Hexabromdiphenylether und Heptabromdiphenylether',
 'Pentachlorophenol\\xc2\\xa0and its salts and esters',
 'pyrimethanil, fluopyram',
 '2,3,4,5-Bis (2-Butylene) Tetrahydro-2-Furaldehyde',
 'Benomylo',
 'Chloromethoxypropylmercuric Acetate',
 'Di(Phenylmercury)Dodecenylsuccinate',
 'Dinitrobutyl Phenol',
 'Dinoseb, Its Acetate And Salts',
 'Ethyl P-Nitrophenyl Thio-Phosphonate',
 'Ethyl Hexyleneglycol',
 'Fosfamidon Formulaci\\xc3\\xb3n L\\xc3\\xadquida Soluble',
 'Heptachlor Y Ep\\xc3\\xb3xido De Heptacloro',
 'Hexachlorociclohexane',
 'Methamidofos',
 'Dodecacloro',
 'Potassium 2,4,5-Trichlorophenate',
 'Tebupirimifos',
 'BENTAZON, SODIUM SALT',
 'ETHEPHON X',
 'NALED X',
 'Verdepryn 100 S',
 'boscalid,',
 'Ceyv']

In [8]:
def fetch_results(search_term, number_results, language_code='en'):
    USER_AGENT = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    assert isinstance(search_term, str), 'Search term must be a string'
    assert isinstance(number_results, int), 'Number of results must be an integer'
    escaped_search_term = search_term.replace(' ', '+')
 
    google_url = 'https://www.google.com/search?q={}&num={}&hl={}'.format(escaped_search_term, number_results, language_code)
    response = requests.get(google_url, headers=USER_AGENT)
    response.raise_for_status()
    print(google_url)
    return search_term, response.text

def parse_results(html, keyword):
    soup = BeautifulSoup(html, 'html.parser')
 
    found_results = []
    rank = 1
    result_block = soup.find_all('div', attrs={'class': 'g'})
    for result in result_block:
 
        link = result.find('a', href=True)
        title = result.find('h3')
        description = result.find('span', attrs={'class': 'st'})
        if link and title:
            link = link['href']
            title = title.get_text()
            if description:
                description = description.get_text()
            if link != '#':
                found_results.append({'keyword': keyword, 'rank': rank, 'title': title, 'description': description, 'link': link})
                rank += 1
    return found_results
def scrape_google(search_term, number_results, language_code='en'):
    try:
        keyword, html = fetch_results(search_term, number_results, language_code)
        results = parse_results(html, keyword)
        return results
    except AssertionError:
        raise Exception("Incorrect arguments parsed to function")
    except requests.HTTPError:
        raise Exception("You appear to have been blocked by Google")
    except requests.RequestException:
        raise Exception("Appears to be an issue with your connection")

In [135]:
def get_from_chembook(name):
    
    print(name)
    time.sleep(30)
    responses=scrape_google(name, 25)
    for r in responses:
        if 'www.chemicalbook.com' in r['link']:
            print(r['link'])
            return scrape_chembook(r['link'])

In [136]:
def scrape_chembook(url):
    response=requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    res_block=soup.find_all('div')
    return check_resblock(res_block)

In [147]:
def get_missing_data(dic):
    results={}
    for name in [k for k,v in dic.items() if v==[]]:
        name=string_fix(name)
        try:
            response=get_from_chembook(name)
            if response:
                results[name]=[response]
            else:
                names=name.split(' ')
                for n in names:
                    if len(n)<3:
                        continue
                    response=get_from_chembook(n)
                    if response:
                        results[n]=response
                    else:
                        results[n]=full_searcher(n)
                    
        except:
            time.sleep(500)
    return results
        

In [151]:
full_searcher('Pentachlorophenoland')

Trying Wikipedia


('Pentachlorophenoland', [])

In [148]:
results=get_missing_data(dic)

Hexabromdiphenylether und Heptabromdiphenylether
https://www.google.com/search?q=Hexabromdiphenylether+und+Heptabromdiphenylether&num=25&hl=en
Hexabromdiphenylether
https://www.google.com/search?q=Hexabromdiphenylether&num=25&hl=en
https://www.chemicalbook.com/ChemicalProductProperty_EN_CB5848454.htm
und
https://www.google.com/search?q=und&num=25&hl=en
Heptabromdiphenylether
https://www.google.com/search?q=Heptabromdiphenylether&num=25&hl=en
https://www.chemicalbook.com/ChemicalProductProperty_EN_CB2500685.htm
Pentachlorophenoland its salts and esters
https://www.google.com/search?q=Pentachlorophenoland+its+salts+and+esters&num=25&hl=en
Pentachlorophenoland
https://www.google.com/search?q=Pentachlorophenoland&num=25&hl=en
Trying Wikipedia
its
https://www.google.com/search?q=its&num=25&hl=en
Trying Wikipedia
salts
https://www.google.com/search?q=salts&num=25&hl=en
Trying Wikipedia
pyrimethanil, fluopyram
https://www.google.com/search?q=pyrimethanil,+fluopyram&num=25&hl=en
pyrimethanil,


In [153]:
os.getcwd()

'/mnt/c/Users/benja/cert_standards/pesticide_lists'

In [155]:
for f in [f for f in os.listdir() if 'txt' in f]:
    if 'bosca' in open(f, encoding='latin1').read().lower():
        print(f)

chemical_ref_nums.txt
trueEarth_restricted.txt


True

In [159]:
response=requests.get('https://www.chemicalbook.com/ProductChemicalPropertiesCB4837684_EN.htm')

In [117]:
 soup = BeautifulSoup(response.text, 'lxml')

In [118]:
res_block=soup.find_all('div')

[<div class="main_960px">
 <div class="top_txt_bg" style="position:relative;font-family:Times New Roman">
 <span id="NoUser" style="display:block;margin-top:1px;vertical-align:middle;margin-bottom:3px;">
 <a href="http://settings.messenger.live.com/Conversation/IMMe.aspx?invitee=7b04453cdd26b12c@apps.messenger.live.com" target="_blank"><font color="#ff3333" style="vertical-align:middle;">Chat Live</font></a> |
 	      <a href="/UserManager/Login_EN.aspx?Lang=CN" onclick="blur()" rel="nofollow" title="Login">Log in</a> | 
 	      <a href="/UserManager/Register_EN.aspx?Lang=CN" onclick="blur()" rel="nofollow">Join Free</a> | 	    	      
 	      <a href="/ProductChemicalPropertiesCB4837684.htm" id="TransLink2"><img id="TransImage2" src="/images/china.bmp"/></a> | 	      	    
 	      <a href="/ProductIndex_JP.aspx" id="TransJapan1"><img id="TransJapanImage1" src="/images/japan.bmp"/></a> | 	   
 	      <a href="/ProductIndex_DE.aspx" id="TransGermany1"><img id="TransGermanyImage1" src="/

In [123]:
check_resblock(res_block)

<div class="main_960px">
<div class="top_txt_bg" style="position:relative;font-family:Times New Roman">
<span id="NoUser" style="display:block;margin-top:1px;vertical-align:middle;margin-bottom:3px;">
<a href="http://settings.messenger.live.com/Conversation/IMMe.aspx?invitee=7b04453cdd26b12c@apps.messenger.live.com" target="_blank"><font color="#ff3333" style="vertical-align:middle;">Chat Live</font></a> |
	      <a href="/UserManager/Login_EN.aspx?Lang=CN" onclick="blur()" rel="nofollow" title="Login">Log in</a> | 
	      <a href="/UserManager/Register_EN.aspx?Lang=CN" onclick="blur()" rel="nofollow">Join Free</a> | 	    	      
	      <a href="/ProductChemicalPropertiesCB4837684.htm" id="TransLink2"><img id="TransImage2" src="/images/china.bmp"/></a> | 	      	    
	      <a href="/ProductIndex_JP.aspx" id="TransJapan1"><img id="TransJapanImage1" src="/images/japan.bmp"/></a> | 	   
	      <a href="/ProductIndex_DE.aspx" id="TransGermany1"><img id="TransGermanyImage1" src="/images/Ge

'18162-48-6'

In [130]:
def check_resblock(res_block):
    for r in res_block:
        if 'CAS' in r.text:
            return re.search('[\d\-]{5,}', r.text).group()
            

In [131]:
check_resblock(res_block)

'18162-48-6'

In [29]:
response=requests.get('https://www.chemicalbook.com/ChemicalProductProperty_EN_CB5848454.htm')

In [31]:
soup=BeautifulSoup(response.text, 'lxml')

In [32]:
soup

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head id="Head1"><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="unsafe-url" name="referrer"/><title>
	2,2',4,4',5,5'-HEXABROMODIPHENYL ETHER | 68631-49-2
</title><meta content="2,2',4,4',5,5'-HEXABROMODIPHENYL ETHER cas no cas number 68631-49-2" name="Keywords"/><meta content="Visit ChemicalBook To find more 2,2',4,4',5,5'-HEXABROMODIPHENYL ETHER(68631-49-2) information like chemical properties,Structure,melting point,boiling point,density,molecular formula,molecular weight, physical properties,toxicity information,customs codes. You can also browse global suppliers,vendor,prices,Price,manufacturers of 2,2',4,4',5,5'-HEXABROMODIPHENYL ETHER(68631-49-2). At last,2,2',4,4',5,5'-HEXABROMODIPHENYL ETHER(68631-49-2) safety, risk, hazard and MSDS, CAS,cas number,Use,cas no may also be you need." name="description"/>
<script src="/Themes/Jquery/jquery-1.11.1.min.js" type="te